# Building a Modular AI Network Assistant with MCP & LangGraph


This notebook demonstrates the implementation of a scalable AI agent using the **Model Context Protocol (MCP)** standard. By leveraging LangGraph for state orchestration and GlobalPing as a remote MCP server, we create a network diagnostic assistant capable of executing real-time connectivity tests (Ping, Traceroute, DNS).

The workflow illustrates how MCP solves the "tool integration" problem by decoupling the agent's logic from the tool's implementation. Key technical concepts covered include:

* **Multi-Server MCP Client**: Connecting a standard LLM to external, standardized tool servers.
* **Asynchronous Execution:** Using Python's asyncio for non-blocking tool usage.
* **State Management:** Orchestrating conversation history and tool outputs with LangGraph.

### Install Depedancies

In [7]:
# !pip install langgraph langchain-openai  nest_asyncio gradio
# !pip install langchain-mcp-adapters

### Import APIs

In [8]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount = True)


# ---------------------------------OpenAI API--------------------------
api_key_file = '/content/drive/MyDrive/api_key.txt'
with open(api_key_file, 'r', encoding = 'utf-8-sig') as file:
    api_key = file.read().strip()

print(api_key is not None)
os.environ['OPENAI_API_KEY'] = api_key
print("OpenAI API key loaded:", os.environ.get('OPENAI_API_KEY') is not None)


# --------------------------------GLOBALPING API------------------------
globalping_token_file = '/content/drive/MyDrive/globalping.txt'  # Adjust the path if necessary

with open(globalping_token_file, 'r') as file:
   globalping_token = file.read().strip()

os.environ['GLOBALPING_TOKEN'] = globalping_token
print("GlobalPing Token loaded:", os.environ.get('GLOBALPING_TOKEN') is not None)

Mounted at /content/drive
True
OpenAI API key loaded: True
GlobalPing Token loaded: True


### Create Network Graph

In [11]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage # Added SystemMessage for robustness
from langchain_mcp_adapters.client import MultiServerMCPClient
import gradio as gr
import os

def create_network_graph(tools):
    # Model setup: using temperature=0 for deterministic tool usage
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
    llm_with_tools = llm.bind_tools(tools=tools)

    async def agent_node(state: MessagesState):
        """
        Process the state and generate a response using the LLM.
        We pass the full history (state['messages']) to maintain context.
        """
        return {"messages": [await llm_with_tools.ainvoke(state["messages"])]}

    tool_node = ToolNode(tools)

    builder = StateGraph(MessagesState)

    # Define Nodes
    builder.add_node('agent', agent_node)
    builder.add_node('tools', tool_node)

    # Define Edges
    builder.add_edge(START, 'agent')
    # tools_condition: Built-in logic that routes to 'tools' if the LLM calls one, else END
    builder.add_conditional_edges('agent', tools_condition)
    builder.add_edge('tools', 'agent')
    builder.add_edge('agent', END)

    # Persistence
    memory = MemorySaver()

    return builder.compile(checkpointer=memory)


async def main():
    print('Initialising MCP Client')

    # MCP Client Configuration
    # 'Multi' allows connecting to multiple distinct servers (e.g., GlobalPing, Slack, Database) simultaneously.
    client = MultiServerMCPClient({
        "globalping": {
            "transport": "http", # Using SSE (Server-Sent Events) over HTTP
            "url": "https://mcp.globalping.dev/mcp",
            "headers": {"Authorization": f"Bearer {os.environ['GLOBALPING_TOKEN']}"}
        }
    })

    print('Fetching tools from MCP Server')
    # Dynamic Tool Loading: No manual wrappers needed. The client fetches definitions from the server.
    tools = await client.get_tools()

    # Optional: Inspect loaded tools
    for tool in tools:
        print(f'Tool Name: {tool.name}')
        print(f'Tool Description: {tool.description[:100]}...')
        print('_' * 30)

    print('Create Network Graph')
    graph = create_network_graph(tools)

    async def interact(message, history):
        config = {'configurable': {'thread_id': 'session1'}}

        # We inject a SystemMessage to guide the model on parameter formatting (avoids API 400 errors)
        inputs = {'messages': [
            SystemMessage(content="You are a network assistant. When using Globalping tools, ensure the 'target' parameter is a simple string (e.g., 'google.com') and not an object."),
            HumanMessage(content=message)
        ]}

        final_state = await graph.ainvoke(inputs, config)
        return final_state['messages'][-1].content


    print('Launching Gradio...')
    demo = gr.ChatInterface(
        fn=interact,
        type='messages',
        title='Network Assistant with Globalping (MCP)'
    )
    demo.queue()
    demo.launch(share=True, debug=True)

In [ ]:
await main()

Initialising MCP Client
Fetching tools from MCP Server
Tool Name: ping
Tool Description: Measure network latency, packet loss, and reachability to a target (domain or IP) from globally dist...
______________________________
Tool Name: traceroute
Tool Description: Trace the network path to a target (domain or IP) from global locations. Use this tool to identify w...
______________________________
Tool Name: dns
Tool Description: Resolve DNS records (A, AAAA, MX, etc.) for a domain from global locations. Use this tool to verify ...
______________________________
Tool Name: mtr
Tool Description: Run an MTR (My Traceroute) diagnostic, which combines Ping and Traceroute. Use this tool to analyze ...
______________________________
Tool Name: http
Tool Description: Send HTTP/HTTPS requests (GET, HEAD or OPTIONS) to a URL from global locations. Use this tool to che...
______________________________
Tool Name: locations
Tool Description: Retrieve the list of available Globalping probe locations